# Crawling data dari pta.trunojoyo.ac.id

In [1]:
_= !pip install bs4

In [2]:
import requests

In [3]:
from bs4 import BeautifulSoup
import pandas as pd

In [4]:
class ScrapingPtaTrunojoyo:
  def __init__(self, url):
    self.url = url
  
  def page(self):
    res = requests.get(self.url)
    soup = BeautifulSoup(res.content, 'html.parser')
    contents = soup.find('ol', class_='pagination').findChildren(recursive=False)
    return int(contents[-3].text)

  def artikel_page(self, page):
    res = requests.get(f'{self.url}/{page}')
    soup = BeautifulSoup(res.content, 'html.parser')
    contents = soup.find_all('li', attrs={'data-cat':'#luxury'})
    return contents

  def get_berita(self):
    pta_list = []
    page = self.page()
    for pg in range(1, page+1):
      artikels = self.artikel_page(pg)
      for artikel in artikels:
        respone = requests.get(artikel.find('a', class_='gray button')['href'])
        soup_data = BeautifulSoup(respone.content, 'html.parser')
        cont = soup_data.find('div', attrs={'id':'content_journal'})
        # ambil data
        nama = cont.findAll('div')[1].div.span.text.split(' : ')[1]
        judul = str.join("", (cont.find('a', class_='title').string).splitlines()).capitalize()
        abstrak = str.join("", (cont.find('p').string).splitlines()).capitalize()
        pta_list.append({
            'Penulis' : nama,
            'Judul': judul,
            'Abstrak': abstrak
        })
    self.pta = pta_list
  
  def result(self):
    return pd.DataFrame(self.pta)
  
  def save(self, namefile='pta'):
    df = pd.DataFrame(self.pta).to_csv(namefile+'.csv', index=False)

In [5]:
scPta = ScrapingPtaTrunojoyo('https://pta.trunojoyo.ac.id/c_search/byprod/10/')

In [6]:
scPta.get_berita()

In [7]:
scPta.result()

,Penulis,Judul,Abstrak
0,A.Ubaidillah S.Kom,Perancangan dan implementasi sistem database t...,Sistem informasi akademik (siakad) merupaka...
1,"M. Basith Ardianto,",Aplikasi kontrol dan monitoring jaringan kompu...,Berjalannya koneksi jaringan komputer dengan l...
2,"Akhmad Suyandi, S.Kom",Rancang bangun aplikasi proxy server untukenkr...,Web server adalah sebuah perangkat lunak serve...
3,Heri Supriyanto,Sistem pendukung keputusan optimasi penjadwala...,Penjadwalan kuliah di perguruan tinggi me...
4,Septian Rahman Hakim,Sistem augmented reality animasi benda bergera...,Seiring perkembangan teknologi yang ada diduni...
5,Adi Chandra Laksono,Gerak pekerja pada game real time strategy men...,Gerak pekerja ada pada game yang memiliki genr...
6,NURRACHMAT,Rancang bangun game perawatan sapi karapan men...,"Perkembangan game yang semakin pesat, memberik..."
7,Muhammad Choirur Rozi,Ekstraksi fitur berbasis two dimensional linea...,Sistem pengenalan wajah adalah suatu sistem un...
8,M Khoiril Anwar,Implementasi algoritma prim dan depth first ...,Teknologi mobile game beroperating system open...
9,MALIKUL HAMZAH,Perancangan sistem informasi badan kepegawaian...,Kantor badan kepegawaian kota bangkalan adalah...


In [8]:
scPta.save()